<a href="https://colab.research.google.com/github/Adeekoa/ETL_practice/blob/main/ca_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import important libraries

In [ ]:
import pandas as pd
import numpy as np
import pypyodbc

### Load the file into a pandas dataframe and preview the dataframe

In [ ]:
df = pd.read_csv('FireBrigadeAndAmbulanceCallOuts.csv')
df.head()

,Date,Station Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD
0,01/01/2013,Rathfarnham,S/S OTHER,01:27:19,01:28:58,01:30:47,01:58:54,02:08:54,02:28:54,02:48:54,03:08:54
1,01/01/2013,Tallaght,Fire CAR,01:49:57,01:50:40,01:51:47,01:55:43,02:05:43,02:25:43,02:45:43,03:05:43
2,01/01/2013,North Strand,S/S RTA,03:35:24,03:36:14,03:38:28,03:41:20,03:51:20,04:11:20,04:31:20,04:51:20
3,01/01/2013,Tallaght,Fire CAR,04:12:51,04:13:56,04:14:32,04:20:48,04:30:48,04:50:48,05:10:48,05:30:48
4,01/01/2013,Finglas,Fire DOM PER,04:14:24,04:15:36,04:17:37,04:20:47,04:30:47,04:50:47,05:10:47,05:30:47


### Output the numbers of rows and columns in the dataset

In [ ]:
df_shape = df.shape
print('The dataset has ', df_shape[0], 'rows', 'and', df_shape[1], 'columns')

The dataset has  38556 rows and 11 columns


### Output the no of non-null rows by columns

In [ ]:
    df.count() #the count function only counts the non-null rows

Date            38556
Station Area    38556
Description     38556
TOC             38556
ORD             38556
MOB             37311
IA              30962
LS              38556
AH                153
MAV             38530
CD              38530
dtype: int64

### Output the number of null values (by column).

In [ ]:
df.isnull().sum()

Date                0
Station Area        0
Description         0
TOC                 0
ORD                 0
MOB              1245
IA               7594
LS                  0
AH              38403
MAV                26
CD                 26
dtype: int64

### Output the number of null values for all columns.

In [ ]:
total_null_values = df.isnull().sum().sum()
print('The total number of rows with null value is', total_null_values)

The total number of rows with null value is 47294


### Output the total number of call outs by Station Area.


In [ ]:
df.groupby(df['Station Area']).size()

Station Area
Balbriggan         688
Blanchardstown    2089
Dolphins Barn     4018
Donnybrook        1787
Dun Laoghaire     3271
Dunshaughlin         2
Finglas           3030
Kilbarrack        3548
MH14                 1
North Strand      1645
Phibsborough      3160
Rathfarnham       2129
Skerries           449
Swords            2614
Tallaght          6525
Tara St           3600
dtype: int64

### Output the total number of call outs by Date and Station Area.

In [ ]:
df.groupby(['Date', 'Station Area']).size()

Date        Station Area  
01/01/2013  Balbriggan        3
            Blanchardstown    1
            Dolphins Barn     4
            Donnybrook        1
            Dun Laoghaire     3
                             ..
31/12/2015  Donnybrook        1
            Dun Laoghaire     1
            Finglas           3
            North Strand      1
            Tallaght          5
Length: 12483, dtype: int64

### Output the total number of call outs by Station Area and Date where the description is either Fire Car or Fire Alarm.


In [ ]:
df_1 = df[df['Description'].isin(['Fire ALARM', 'Fire Car'])]
df_1.groupby(['Station Area', 'Date']).size()

Station Area  Date      
Balbriggan    01/01/2013    2
              01/01/2014    1
              02/03/2013    1
              02/06/2013    1
              02/11/2013    1
                           ..
Tara St       31/07/2015    2
              31/08/2013    1
              31/10/2014    1
              31/10/2015    1
              31/12/2013    1
Length: 5236, dtype: int64

### Replace any instance of “,” (in any column) with an empty string

In [ ]:
df.replace(",", "",inplace=True)

### Replace any instance of “-” (in any column) with an empty string.

In [ ]:
df.replace("-","", inplace = True)

### Drop rows for the columns (AH, MAV, CD) where at least one row value is NULL.

In [ ]:
df.dropna(subset=['CD','AH','MAV'],inplace=True)

### Drop any duplicate rows (except for the first occurrence).

In [ ]:
df.drop_duplicates(keep='first',inplace=True)

### Output the minimum time difference between TOC and ORD.

In [ ]:
#step 1: Merge the date column with both TOC and ORD

TOC_Date = df['Date']+" " +df['TOC']
ORD_Date = df['Date'] +" "+df['ORD']


In [ ]:
#Step 2: convert the merged columns into a datetime object

TOC_Date=pd.to_datetime(TOC_Date)
ORD_Date=pd.to_datetime(ORD_Date)

In [ ]:
#Step 3: find the minimum difference between TOC_Date and ORD_Date

min_response_time = min(ORD_Date - TOC_Date)
print('The minimum response time is', min_response_time)

The minimum response time is 0 days 00:00:30


### Further Data Cleaning 

In [ ]:
#Rename columns in line with convention
df.rename({'Station Area':'Station_Area'}, inplace = True, axis = 1)

In [ ]:
#Replace missing values with 'None'
df=df.fillna('None')

### Connect to MS SQL Server

In [ ]:
connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
'Server=sni;'
'Database=master;'
'encrypt=yes;'
'TrustServerCertificate=yes;'
'UID=sa;'
'PWD=Doctor27$',autocommit = True)

### Create a table with the pandas dataframe attributes on the SQL Server database

In [ ]:
cursor = connection.cursor()
SQLCommand = ("""CREATE TABLE Accident_response (Date varchar (256),
            Station_Area varchar (256),
            Description varchar (256),
            TOC varchar (256),
            ORD varchar (256) ,
            MOB varchar (256),
            IA varchar (256),
            LS varchar (256),
            AH varchar (256),
            MAV varchar (256),
            CD varchar (256)) 
              """)
cursor.execute(SQLCommand)
print('done')

done


### Insert the rows into the created table on sql server

In [ ]:
for index, row in df.iterrows():
    #print (index)
    #if index>6:
       #print(row)
    query = "INSERT INTO Accident_response (Date, Station_Area, Description, TOC, ORD, MOB, IA, LS, AH, MAV, CD) VALUES(?,?,?,?,?,?,?,?,?,?,?)"
    cursor.execute(query, (row.Date, row.Station_Area, row. Description, row.TOC, row.ORD, row.MOB, row.IA, row.LS, row.AH, row.MAV, row.CD))

### Check if the table is update on the sql server

In [ ]:
query = "SELECT * from Accident_response;"
df1 = pd.read_sql(query, connection)
df1.head(10)

,date,station_area,description,toc,ord,mob,ia,ls,ah,mav,cd
0,01/01/2013,Rathfarnham,S/S OTHER,01:27:19,01:28:58,01:30:47,01:58:54,02:08:54,02:28:54,02:48:54,03:08:54
1,01/01/2013,Tallaght,Fire CAR,01:49:57,01:50:40,01:51:47,01:55:43,02:05:43,02:25:43,02:45:43,03:05:43
2,01/01/2013,North Strand,S/S RTA,03:35:24,03:36:14,03:38:28,03:41:20,03:51:20,04:11:20,04:31:20,04:51:20
3,01/01/2013,Tallaght,Fire CAR,04:12:51,04:13:56,04:14:32,04:20:48,04:30:48,04:50:48,05:10:48,05:30:48
4,01/01/2013,Finglas,Fire DOM PER,04:14:24,04:15:36,04:17:37,04:20:47,04:30:47,04:50:47,05:10:47,05:30:47
5,01/01/2013,Tallaght,S/S OTHER,04:54:30,04:59:08,05:02:07,05:09:48,05:19:48,05:39:48,05:59:48,06:19:48
6,01/01/2013,Dun Laoghaire,Fire ALARM,05:12:03,05:15:00,05:17:23,05:25:13,05:35:13,05:55:13,06:15:13,06:35:13
7,01/01/2013,Tara St,Fire ALARM,05:45:45,05:47:55,05:51:19,None,00:10:00,00:30:00,00:50:00,01:10:00
8,01/01/2013,Balbriggan,Fire ALARM,05:59:23,06:00:13,06:03:34,06:10:10,06:20:10,06:40:10,07:00:10,07:20:10
9,01/01/2013,Kilbarrack,Fire ALARM,06:07:42,06:08:51,06:10:41,06:13:47,06:23:47,06:43:47,07:03:47,07:23:47


In [ ]:
print("Transformed dataframe:",df.shape,"\n","SQL dataframe:", df1.shape)

Transformed dataframe: (152, 11) 
 SQL dataframe: (152, 11)


In [ ]:
cursor = connection.cursor()
SQLCommand = ("""DROP TABLE python_table1
              """)
cursor.execute(SQLCommand)
print('done')


done


In [ ]:
#from sqlalchemy.engine import URL
#connection_url = URL.create(
#    "mssql+pyodbc",
#    username="sa",
#    password="Doctor27$",
#    host="20.107.9.62",
#    port=49242,
#    database="master",
#    query={
#        "driver": "ODBC Driver 17 for SQL Server",
#        "authentication": "ActiveDirectoryIntegrated",
#    },
#)

In [ ]:
#from sqlalchemy.engine import URL
#from sqlalchemy import create_engine
#connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=20.107.9.62;DATABASE=master;UID=sa;PWD=Doctor27$"
#connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
#
#engine = create_engine(connection_url)
#
#df.to_sql("Accident_response_table", engine)